# Data Gathering

This notebook contains information on downloading market (end of day price stock prices) data, fundamental data,  alternative data from different sources for the stocks part listed in the S&P 500 index.



## 1. Setup
Let's start with installing a few libraries that we are going to use and setting up our environment:

### 1.1 Install, import and setup libraries

In [1]:
%pip install eod
%pip install yfinance
%pip install tables
%pip install fmp_python
%matplotlib inline


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import bs4 as bs
import certifi
from datetime import date
from eod import EodHistoricalData
from fmp_python.fmp import FMP
from functools import reduce
from io import BytesIO
import json
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np
import pandas as pd
import pandas_datareader.data as web
from pathlib import Path
from pprint import pprint
import requests
import seaborn as sns
import ssl
import time
from urllib.request import urlopen
import yfinance as yf
from zipfile import ZipFile, BadZipFile

pd.set_option('display.expand_frame_repr', False)

### 1.2 Set Paths

Currently we will keep our data in the __/data__ folder in which the current file resides. Modify path if you would like to store the data elsewhere and change the notebooks accordingly.

In [2]:
data_path = Path('data') # perhaps set to external harddrive to accomodate large amount of data
if not data_path.exists():
    data_path.mkdir()
data_store = Path('data/data.h5')

### 1.3 Set API variables

In [6]:
#API KEY from https://site.financialmodelingprep.com/d
API_KEY = '6ee1ddf9fe496202eaf02b9b6fd6e396'

#init dataframes
sp500_constituents = pd.DataFrame()
historical_prices = pd.DataFrame()
ratios = pd.DataFrame()
balance_sheet_statements = pd.DataFrame()
income_statements = pd.DataFrame()
cash_flow_statements = pd.DataFrame()

In [7]:
#init api urls
historical_prices_api = 'https://financialmodelingprep.com/api/v3/historical-price-full'
ratios_api = 'https://financialmodelingprep.com/api/v3/ratios'
balance_sheet_statements_api = 'https://financialmodelingprep.com/api/v3/balance-sheet-statement'
income_statements_api = 'https://financialmodelingprep.com/api/v3/income-statement'
cash_flow_statements_api = 'https://financialmodelingprep.com/api/v3/cash-flow-statement'
sp500_constituent_api = 'https://financialmodelingprep.com/api/v3/sp500_constituent'

### 1.4 Create helper function

In [8]:
def get_data(url_api, ticker, period , limit, key, data_type):
    if data_type == 'f':
        url = (f'{url_api}/{ticker}?period={period}&limit={limit}&apikey={key}')
    else:
        url = (f'{url_api}/{ticker}?apikey={key}')
    context = ssl.create_default_context(cafile=certifi.where())
    response = urlopen(url, context = context)
    return pd.read_json(response.read().decode("utf-8"))

## 2. Collecting S&P 500 component stocks

For benchmark we will compare the results of our automated trading system by creating a portfolio with the stocks part of the index S&P 500 with the results of the index itself.

The following code downloads the S&P500 stocks lists.

In [22]:
url = (f'{sp500_constituent_api}?apikey={API_KEY}')
context = ssl.create_default_context(cafile=certifi.where())
response = urlopen(url, context = context)
sp500_constituents = pd.read_json(response.read().decode("utf-8"))
sp500_constituents.head()

,symbol,name,sector,subSector,headQuarter,dateFirstAdded,cik,founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


Finding odd entries in tickers list, these tickers do not have data from the scrape and do not

In [23]:
bad_tickers = []
for ticker in sp500_constituents.symbol:
    if '.' in ticker:
        bad_tickers.append(ticker)
        
# Exclude the bad tickers from our list
sp500_constituents = sp500_constituents[~sp500_constituents.symbol.isin(bad_tickers)]
#Creare index
sp500_constituents = sp500_constituents.set_index('symbol')
sp500_constituents['dateFirstAdded'] = pd.to_datetime(sp500_constituents.dateFirstAdded).dt.tz_localize(None)

In [24]:
sp500_constituents.info()

<class 'pandas.core.frame.DataFrame'>
Index: 503 entries, MMM to ZTS
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   name            503 non-null    object        
 1   sector          503 non-null    object        
 2   subSector       503 non-null    object        
 3   headQuarter     503 non-null    object        
 4   dateFirstAdded  459 non-null    datetime64[ns]
 5   cik             503 non-null    int64         
 6   founded         503 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 31.4+ KB


## 3. Get market and fundamental data

In [10]:
for ticker in sp500_constituents.index:
    print(f'Loading data for index {ticker}')
    #historical prices
    tmp_historical_prices_df = get_data(historical_prices_api, ticker, '',20, API_KEY,'')
    tmp_historical_prices_df = pd.concat([tmp_historical_prices_df, pd.json_normalize(tmp_historical_prices_df.pop("historical"))], axis=1)
    historical_prices = pd.concat([historical_prices,tmp_historical_prices_df])

    #ratios
    tmp_ratios_df = get_data(ratios_api, ticker, 'quarter',20, API_KEY,'f')
    ratios = pd.concat([ratios,tmp_ratios_df])

    #balance sheet statements
    tmp_balance_sheet_statements_df = get_data(balance_sheet_statements_api, ticker, 'quarter',20, API_KEY,'f')
    balance_sheet_statements= pd.concat([balance_sheet_statements,tmp_balance_sheet_statements_df])
    
    #cash flow statements
    tmp_cash_flow_statements_df = get_data(cash_flow_statements_api, ticker, 'quarter',20, API_KEY,'f')
    cash_flow_statements = pd.concat([cash_flow_statements,tmp_cash_flow_statements_df])
    
    #income statements
    tmp_income_statements_df = get_data(income_statements_api, ticker, 'quarter',20, API_KEY,'f')
    income_statements = pd.concat([income_statements,tmp_income_statements_df])


Loading data for index MMM
Loading data for index AOS
Loading data for index ABT
Loading data for index ABBV
Loading data for index ACN
Loading data for index ATVI
Loading data for index ADM
Loading data for index ADBE
Loading data for index ADP
Loading data for index AAP
Loading data for index AES
Loading data for index AFL
Loading data for index A
Loading data for index APD
Loading data for index AKAM
Loading data for index ALK
Loading data for index ALB
Loading data for index ARE
Loading data for index ALGN
Loading data for index ALLE
Loading data for index LNT
Loading data for index ALL
Loading data for index GOOGL
Loading data for index GOOG
Loading data for index MO
Loading data for index AMZN
Loading data for index AMCR
Loading data for index AMD
Loading data for index AEE
Loading data for index AAL
Loading data for index AEP
Loading data for index AXP
Loading data for index AIG
Loading data for index AMT
Loading data for index AWK
Loading data for index AMP
Loading data for ind

In [11]:
income_statements.info()

,date,symbol,reportedCurrency,cik,fillingDate,acceptedDate,calendarYear,period,revenue,costOfRevenue,...,incomeBeforeTaxRatio,incomeTaxExpense,netIncome,netIncomeRatio,eps,epsdiluted,weightedAverageShsOut,weightedAverageShsOutDil,link,finalLink
15,2018-12-31,ZTS,USD,1555280.0,2019-02-14,2019-02-14 17:08:33,2018,Q4,1564000000,544000000,...,0.267263,73000000,345000000,0.220588,0.72,0.71,480603000,484479000,https://www.sec.gov/Archives/edgar/data/155528...,https://www.sec.gov/Archives/edgar/data/155528...
16,2018-09-30,ZTS,USD,1555280.0,2018-11-01,2018-11-01 16:43:32,2018,Q3,1480000000,473000000,...,0.282432,71000000,347000000,0.234459,0.72,0.71,482000000,485800000,https://www.sec.gov/Archives/edgar/data/155528...,https://www.sec.gov/Archives/edgar/data/155528...
17,2018-06-30,ZTS,USD,1555280.0,2018-08-02,2018-08-02 15:34:20,2018,Q2,1415000000,447000000,...,0.308834,55000000,384000000,0.271378,0.79,0.79,483800000,487500000,https://www.sec.gov/Archives/edgar/data/155528...,https://www.sec.gov/Archives/edgar/data/155528...
18,2018-03-31,ZTS,USD,1555280.0,2018-05-02,2018-05-02 08:08:42,2018,Q1,1366000000,447000000,...,0.305271,67000000,352000000,0.257687,0.72,0.72,485900000,489800000,https://www.sec.gov/Archives/edgar/data/155528...,https://www.sec.gov/Archives/edgar/data/155528...
19,2017-12-31,ZTS,USD,1555280.0,2018-02-15,2018-02-15 12:31:15,2017,Q4,1460000000,457000000,...,0.294521,350000000,81000000,0.055479,0.17,0.16,487323000,491022000,https://www.sec.gov/Archives/edgar/data/155528...,https://www.sec.gov/Archives/edgar/data/155528...


In [51]:
ratios['calendarYear'] = ratios['date'].dt.year
ratios.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9963 entries, 0 to 19
Data columns (total 58 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   symbol                              9963 non-null   object        
 1   date                                9963 non-null   datetime64[ns]
 2   period                              9963 non-null   object        
 3   currentRatio                        9459 non-null   float64       
 4   quickRatio                          9459 non-null   float64       
 5   cashRatio                           9459 non-null   float64       
 6   daysOfSalesOutstanding              9952 non-null   float64       
 7   daysOfInventoryOutstanding          8798 non-null   float64       
 8   operatingCycle                      8797 non-null   float64       
 9   daysOfPayablesOutstanding           8798 non-null   float64       
 10  cashConversionCycle       

In [23]:
balance_sheet_statements.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9991 entries, 0 to 19
Data columns (total 54 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   date                                     9991 non-null   datetime64[ns]
 1   symbol                                   9991 non-null   object        
 2   reportedCurrency                         9991 non-null   object        
 3   cik                                      9988 non-null   float64       
 4   fillingDate                              9991 non-null   object        
 5   acceptedDate                             9991 non-null   object        
 6   calendarYear                             9991 non-null   int64         
 7   period                                   9991 non-null   object        
 8   cashAndCashEquivalents                   9991 non-null   float64       
 9   shortTermInvestments                     99

In [16]:
historical_prices.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 626773 entries, 0 to 1256
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   symbol            626773 non-null  object 
 1   date              626773 non-null  object 
 2   open              626773 non-null  float64
 3   high              626773 non-null  float64
 4   low               626773 non-null  float64
 5   close             626773 non-null  float64
 6   adjClose          626773 non-null  float64
 7   volume            626773 non-null  int64  
 8   unadjustedVolume  626773 non-null  int64  
 9   change            626773 non-null  float64
 10  changePercent     626773 non-null  float64
 11  vwap              626773 non-null  float64
 12  label             626773 non-null  object 
 13  changeOverTime    626773 non-null  float64
dtypes: float64(9), int64(2), object(3)
memory usage: 71.7+ MB


## 3. Save data

### 3.1 Save data to CSV

In [26]:
def save_to_csv(df,name):
    df = df.reset_index()
    if 'index' in df.columns:
          df = df.drop(['index'], axis = 1)
    df['ticker'] = df['symbol']
    df = df.drop(['symbol'], axis = 1)
    df.to_csv(f'data/{name}.csv')

In [60]:
save_to_csv(historical_prices,'historical_prices')
save_to_csv(balance_sheet_statements,'balance_sheet_statements')
save_to_csv(ratios,'ratios')
save_to_csv(income_statements,'income_statements')
save_to_csv(cash_flow_statements,'cash_flow_statements')
save_to_csv(sp500_constituents,'sp500_constituents')

In [27]:
save_to_csv(sp500_constituents,'sp500_constituents')

### 3.2 Save to HDF

In [28]:
def save_to_hdf(name):
    df = (pd.read_csv(f'data/{name}.csv',
                 parse_dates=['date'],
                 index_col=['date', 'ticker'],
                 infer_datetime_format=True)
     .sort_index())
    with pd.HDFStore(data_store) as store:
        store.put(f'stocks/{name}', df)

In [29]:
save_to_hdf('historical_prices')
save_to_hdf('balance_sheet_statements')
save_to_hdf('ratios')
save_to_hdf('income_statements')
save_to_hdf('cash_flow_statements')
with pd.HDFStore(data_store) as store:
        store.put('stocks/sp500_constituents', sp500_constituents)

/var/folders/hp/29p2m8ms4vg51dy79knkbqgh0000gn/T/ipykernel_79646/2150501944.py:8: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['reportedCurrency', 'fillingDate', 'acceptedDate', 'period', 'link',
       'finalLink'],
      dtype='object')]

  store.put(f'stocks/{name}', df)
/var/folders/hp/29p2m8ms4vg51dy79knkbqgh0000gn/T/ipykernel_79646/2150501944.py:8: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['reportedCurrency', 'fillingDate', 'acceptedDate', 'period', 'link',
       'finalLink'],
      dtype='object')]

  store.put(f'stocks/{name}', df)
/var/folders/hp/29p2m8ms4vg51dy79knkbqgh0000gn/T/ipykernel_79646/2150501944.py:8: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it 

In [ ]:
sp500_constituents

,symbol,name,sector,subSector,headQuarter,dateFirstAdded,cik,founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
498,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
499,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
500,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
501,ZION,Zions Bancorporation,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


In [13]:
income_statements.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Empty DataFrame


In [73]:
cash_flow_statements.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10006 entries, 0 to 19
Data columns (total 40 columns):
 #   Column                                    Non-Null Count  Dtype         
---  ------                                    --------------  -----         
 0   date                                      10006 non-null  datetime64[ns]
 1   symbol                                    10006 non-null  object        
 2   reportedCurrency                          10006 non-null  object        
 3   cik                                       10003 non-null  float64       
 4   fillingDate                               10006 non-null  object        
 5   acceptedDate                              10006 non-null  object        
 6   calendarYear                              10006 non-null  int64         
 7   period                                    10006 non-null  object        
 8   netIncome                                 10006 non-null  float64       
 9   depreciationAndAmortization    

In [74]:
balance_sheet_statements.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9991 entries, 0 to 19
Data columns (total 54 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   date                                     9991 non-null   datetime64[ns]
 1   symbol                                   9991 non-null   object        
 2   reportedCurrency                         9991 non-null   object        
 3   cik                                      9988 non-null   float64       
 4   fillingDate                              9991 non-null   object        
 5   acceptedDate                             9991 non-null   object        
 6   calendarYear                             9991 non-null   int64         
 7   period                                   9991 non-null   object        
 8   cashAndCashEquivalents                   9991 non-null   float64       
 9   shortTermInvestments                     99

In [75]:
ratios.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9963 entries, 0 to 19
Data columns (total 58 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   symbol                              9963 non-null   object        
 1   date                                9963 non-null   datetime64[ns]
 2   period                              9963 non-null   object        
 3   currentRatio                        9459 non-null   float64       
 4   quickRatio                          9459 non-null   float64       
 5   cashRatio                           9459 non-null   float64       
 6   daysOfSalesOutstanding              9952 non-null   float64       
 7   daysOfInventoryOutstanding          8798 non-null   float64       
 8   operatingCycle                      8797 non-null   float64       
 9   daysOfPayablesOutstanding           8798 non-null   float64       
 10  cashConversionCycle       